# Goodness of fit test for Uniformity: Validation

**Author:** Nogarbek Sharykpayev  
**Date:** 22.07.2025  
**Based on:** A new omnibus test of fit based on a characterization of the uniform distribution
B. Ebner, **S**. C. Liebenberg & I. J. H. Visagie

## This notebook presents a compact mathematical proof for the numerical approximation of an omnibus test statistic for uniformity testing, along with Monte Carlo validation that reproduces the original research results that can be found here https://www.tandfonline.com/doi/abs/10.1080/02331888.2022.2133121.


## 1. Problem statement

### 1.1 Research Gap
The authors of the original article "A new omnibus test of fit based on a characterisation of the uniform distribution" proposed a goodness-of-fit test for uniformity based on the conditional expectation property of uniform distribution:

 Let $U$ be a random variable taking values in $(0,1)$. Then $U \sim \mathcal{U}$ if, and only if,
$$\mathbb{E}[U | U \geq t] = \frac{1}{2} \bigg( 1 + \frac{t(1-t)}{\mathbb{P}(U \geq t)} \bigg),\: \: \: 0 < t <1,$$
where $\mathcal{U}$ is the standard uniform distribution.



 While they provided the final numerical approximation for their test statistic, they did not include the complete mathematical derivation showing how to transform from the integral form to the computationally feasible sum form.

### 1.2 Specific Problems Addressed
This notebook addresses two things:

Mathematical Proof: Provide a derivation showing the transformation:

$$
\begin{aligned}
T_n &= n \int_0^1 \left( \frac{1}{n} \sum_{j=1}^{n} (2U_j - 1)\mathbb{1}_{\{ U_j \geq t \}} - t(1 - t) \right)^2 dt \\
&= \frac{1}{n} \sum_{j=1}^n \sum_{k=1}^n (4U_jU_k - 2(U_j + U_k) + 1)\min(U_j,U_k) \\
&\quad - \frac{1}{3} \sum_{j=1}^n (2U_j - 1)U_j^2(3 - 2U_j) + \frac{n}{30}.
\end{aligned}
$$



Validation: This notebook provides Monte Carlo Simulation to confirm the critival values presented in the article.

### 1.3 Objectives

First, derive the mathematical transformation from integral to numerical form.

Second, implement the test statistic calculation in Python,
reproduce the critical values using Monte Carlo simulation and finally
validate our results against the original research findings


## 2. Mathematical Proof

### 2.1 Proof Steps Outline

First, we place $\frac{1}{n}$ outside of the integral and apply the algebraic identity $(\sum_{j=1}^n |a_j|)^2 = \sum_{j=1}^n \sum_{k=1}^n | a_j a_k |$ to transform the squared sum into a double sum.

Next, we multiply out the brackets inside the integral, which results in three distinct terms. We simplify terms involving sums over $j$ and $k$ where indices are independent, utilizing the property that $\sum_{j=1}^n a_j + \sum_{k=1}^n a_k = 2 \sum_{j=1}^n a_j$. Terms that are independent of the summation index are treated as constants and converted into a product of the term with $n$ (or $n^2$ for double sums).

Subsequently, we can swap the integral with the finite series (summation) due to the linearity of integration and the finite nature of the series, allowing us to apply the Lebesgue integration. Finally, we multiply the leading $\frac{1}{n}$ factor with all three resulting terms to obtain the simplified expression.

---
### 2.2 Properties of Indicator Functions and Min-Function

The proofs below make use of the $\min$ function, which takes the smallest value among its arguments. For example, $\min(x,y) = y$ if $x > y$.

Furthermore, we utilize the following key equalities for indicator functions $\mathbf{1}_{\{\cdot\}}$ and integrals:

1.  
$$
\mathbf{1}_{\{U_j \geq t\}} \cdot \mathbf{1}_{\{ U_k \geq t\}} = \mathbf{1}_{\{\min(U_j,U_k) \geq t\}}
$$

This equality holds because the product of two indicator functions is 1 if and only if both conditions are met. For $\mathbf{1}_{\{U_j \geq t\}} \cdot \mathbf{1}_{\{ U_k \geq t\}}$ to be 1, both $U_j \geq t$ and $U_k \geq t$ must be true. This implies that $t$ must be less than or equal to both $U_j$ and $U_k$, which is equivalent to $t \leq \min(U_j, U_k)$. Hence, $\mathbf{1}_{\{\min(U_j,U_k) \geq t\}} = 1$.  

Without loss of generality, assume $U_j \geq U_k$. If $U_k < t < U_j$, then $\mathbf{1}_{\{U_k \geq t\}} = 0$, making the product zero, which is consistent with $\mathbf{1}_{\{\min(U_j, U_k) \geq t\}} = \mathbf{1}_{\{U_k \geq t\}} = 0$.

2.  
$$
\int_0^1 \mathbf{1}_{\{U \geq t \}} \, dt = \int_0^{U} 1 \, dt = U
$$

This equation holds because the indicator function $\mathbf{1}_{\{U \geq t\}}$ equals 1 only when $t \leq U$, and 0 otherwise. Thus, integrating $\mathbf{1}_{\{U \geq t\}}$ from 0 to 1 is equivalent to integrating 1 from 0 to $U$.


---


### 2.3 Detailed Transformation

Let's proceed with the detailed transformation of the test statistic $T_n$:

$$
\begin{align*}
T_n &= n \int_0^1 \left( \frac{1}{n} \sum_{j=1}^{n} (2U_j - 1)\mathbf{1}_{\{ U_j \geq t \}} - t(1 - t) \right)^2 dt \\
&= \frac{1}{n} \int_0^1 \sum_{j=1}^n \sum_{k=1}^n \left[ (2U_j -1)\mathbf{1}_{\{ U_j \geq t \}} - t(1-t)\right] \cdot \left[ (2U_k -1)\mathbf{1}_{\{ U_k \geq t \}} - t(1-t)\right] dt \\
&= \frac{1}{n} \int_0^1 \Bigg[\sum_{j=1}^n \sum_{k=1}^n (2U_j - 1)(2U_k - 1)\mathbf{1}_{\{ U_j \geq t \}} \mathbf{1}_{\{ U_k \geq t \}} \\
&\quad - \sum_{j=1}^n \sum_{k=1}^n (2U_j - 1) \mathbf{1}_{\{ U_j \geq t \}} t(1-t) \\
&\quad - \sum_{j=1}^n \sum_{k=1}^n (2U_k - 1) \mathbf{1}_{\{ U_k \geq t \}} t(1-t) \\
&\quad + \sum_{j=1}^n \sum_{k=1}^n t^2(1-t)^2 \Bigg] dt \\
&= \frac{1}{n} \int_0^1 \Bigg[\sum_{j=1}^n \sum_{k=1}^n \big(4U_jU_k - 2(U_j + U_k) + 1\big) \mathbf{1}_{\{ \min(U_j,U_k) \geq t \}} \\
&\quad - 2n \sum_{j=1}^n (2U_j - 1) \mathbf{1}_{\{ U_j \geq t \}} t(1-t) + n^2 t^2(t^2 - 2t + 1) \Bigg] dt \\
&= \frac{1}{n} \Bigg[ \sum_{j=1}^n \sum_{k=1}^n \big(4U_jU_k - 2(U_j + U_k) + 1\big) \int_0^{\min(U_j,U_k)} 1 dt \\
&\quad - 2n \sum_{j=1}^n (2U_j - 1) \int_0^{U_j} t(1 - t) dt + n^2 \int_0^1 t^2(t^2 - 2t + 1) dt \Bigg] \\
&= \frac{1}{n} \Bigg( \sum_{j=1}^n \sum_{k=1}^n \big(4U_jU_k - 2(U_j + U_k) + 1\big) \min(U_j,U_k) \\
&\quad - 2n \sum_{j=1}^n (2U_j - 1) \left[\frac{t^2}{2} - \frac{t^3}{3}\right]_0^{U_j} + n^2 \left[\frac{t^5}{5} - \frac{t^4}{2} + \frac{t^3}{3}\right]_0^1 \Bigg) \\
&= \frac{1}{n} \sum_{j=1}^n \sum_{k=1}^n \big(4U_jU_k - 2(U_j + U_k) + 1\big) \min(U_j,U_k) \\
&\quad - 2 \sum_{j=1}^n (2U_j - 1) \left(\frac{3U_j^2 - 2U_j^3}{6}\right) + n \left(\frac{6 - 15 + 10}{30}\right) \\
&= \frac{1}{n} \sum_{j=1}^n \sum_{k=1}^n \big(4U_jU_k - 2(U_j + U_k) + 1\big) \min(U_j,U_k) \\
&\quad - \frac{1}{3} \sum_{j=1}^n (2U_j - 1) U_j^2(3 - 2U_j) + \frac{n}{30}.
\end{align*}
$$

---

### 2.4 Final Numerical Form
The resulting test statistic is:

$$
\begin{align*}
T_n &= \frac{1}{n} \sum_{j=1}^n \sum_{k=1}^n \big(4U_jU_k - 2(U_j + U_k) + 1\big) \min(U_j,U_k) \\
&\quad - \frac{1}{3} \sum_{j=1}^n (2U_j - 1) U_j^2(3 - 2U_j) + \frac{n}{30}.
\end{align*}
$$


## 3 Code

In [2]:
import numpy as np


def calculate_test_statistic(n):
  """
    Calculates the test statistic T_n based on
    n standard uniform random variables U_1,...,U_n.

    Args:
        n (int): The sample size (dimension of the finite Hilbert vector space).

    Returns:
        float: The calculated value of the test statistic T_n.
  """

  # Generates array with n realizations of standard uniform distribution
  U = np.random.uniform(0,1,n)

  # Create matrices for all pairs (j,k)
  U_j = U[:, None]   # shape (n,1)
  U_k = U[None, :]   # shape (1,n)

  # Vectorized computation of min(U_j, U_k)
  min_U = np.minimum(U_j, U_k)  # shape (n,n)

  # Vectorized first term
  term1_matrix = (4 * U_j * U_k - 2 * (U_j + U_k) + 1) * min_U
  term1 = np.sum(term1_matrix) / n

  # Vectorized second term
  term2 = np.sum((2 * U - 1) * U**2 * (3 - 2 * U)) / 3

  # Third term is constant for given n
  term3 = n / 30

  T_n = term1 - term2 + term3
  return T_n

def monte_carlo_simulation(n,m,alpha):
  """
  This function performs a Monte Carlo simulation
  to approximate the critical value of a test statistic under the null hypothesis.

  Args:
      n (int): The sample size (dimension of the finite Hilbert vector space).
      m (int): The number of simulations.
      alpha (float): significance level
  """
  # to duplicate same results
  np.random.seed(42)


  simulations = [calculate_test_statistic(n) for _ in range(m)]

  #calculate critical value for each simulation step
  critical_value = np.percentile(simulations, 100 * (1 - alpha))

  print(f"Critical value at {(1 - alpha) * 100:.0f}%: {critical_value}")

## 4. Validation

In this final step we will present the results of our designed code and compare it to the results from the article

In [3]:
# Define vectors of parameters as in your table
n_values = [10, 20, 30, 50, 75, 100, 150, 200, 300, 500]
alpha_values = [0.01, 0.05, 0.1]
m = 100000  # Number of simulations

# Store results in a dictionary or structured array
results = {}

for alpha in alpha_values:
    crit_vals_for_alpha = []
    print(f"Running simulations for alpha = {alpha}")
    for n in n_values:
        # Run Monte Carlo simulation to get critical value
        simulations = [calculate_test_statistic(n) for _ in range(m)]
        critical_value = np.percentile(simulations, 100 * (1 - alpha))
        crit_vals_for_alpha.append(critical_value)
        print(f"n = {n}, Critical value = {critical_value:.4f}")
    results[alpha] = crit_vals_for_alpha

# Optionally convert results to DataFrame for pretty printing
import pandas as pd
df = pd.DataFrame(results, index=n_values)
df.index.name = 'n'
print("\nCritical values table:")
print(df)

Running simulations for alpha = 0.01
n = 10, Critical value = 0.7695
n = 20, Critical value = 0.7699
n = 30, Critical value = 0.7739
n = 50, Critical value = 0.7791
n = 75, Critical value = 0.7914
n = 100, Critical value = 0.7887
n = 150, Critical value = 0.7821
n = 200, Critical value = 0.7877
n = 300, Critical value = 0.7925
n = 500, Critical value = 0.7808
Running simulations for alpha = 0.05
n = 10, Critical value = 0.4554
n = 20, Critical value = 0.4550
n = 30, Critical value = 0.4598
n = 50, Critical value = 0.4575
n = 75, Critical value = 0.4593
n = 100, Critical value = 0.4599
n = 150, Critical value = 0.4629
n = 200, Critical value = 0.4590
n = 300, Critical value = 0.4615
n = 500, Critical value = 0.4694
Running simulations for alpha = 0.1
n = 10, Critical value = 0.3292
n = 20, Critical value = 0.3277
n = 30, Critical value = 0.3315
n = 50, Critical value = 0.3290
n = 75, Critical value = 0.3322
n = 100, Critical value = 0.3327
n = 150, Critical value = 0.3294
n = 200, Criti